In [22]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from PIL import Image
# For utilities
import os, shutil, time

In [23]:
%load_ext tensorboard
%tensorboard --logdir=runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 2904), started 23:44:36 ago. (Use '!kill 2904' to kill it.)

In [25]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [24]:
import csv

embeddings = {}
with open("../embeddings-nosoft.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        filename = row[0]
        values = [float(val) for val in row[1:]]
        embeddings[filename] = values


In [26]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [27]:
SIZE = 64
import os
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(), 
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE),
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        filename = os.path.basename(self.paths[index])
        embd = np.array(embeddings[filename])
        embd = torch.from_numpy(embd).unsqueeze(0).float()
        return img_gray, img_ab, embd
    
    def __len__(self):
        return len(self.paths)

In [28]:
# Training
batch_size = 128
train_imagefolder = LabImageFolder('../../datasets/dataset/train/colour')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder('../../datasets/dataset/test/colour' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [29]:
class FusionLAyer(nn.Module):
    def __init__(self):
        super().__init__()
        pass

    def forward(self, imgs, embds):
        embds = torch.reshape(embds, (embds.shape[0], embds.shape[2]))
        reshaped_shape = (imgs.shape[0], embds.shape[1], imgs.shape[2], imgs.shape[3])
        #shapetosave = (embds.shape[0], imgs.shape[2] * imgs.shape[3], embds.shape[1])
        embds = embds.repeat(1,imgs.shape[2] * imgs.shape[3]) #shape (128, 64000)
        #embds = torch.reshape(embds, shapetosave)
        #embds = torch.reshape(embds, reshaped_shape)
        embds = torch.reshape(embds, reshaped_shape)
        cos = torch.cat((imgs, embds), dim=1)
        return cos
      

In [31]:
kernel_size=3
stride_en=2
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
class ColorizationNet(nn.Module):
  def __init__(self, input_size=128):
    super(ColorizationNet, self).__init__()
    
    self.encoder = nn.Sequential(       
      nn.Conv2d(1, 16, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.Conv2d(16, 32, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32, 64, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(64),
      nn.ReLU(),   
    )     
        
    self.decoder = nn.Sequential(  
      nn.ConvTranspose2d(64, 32, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Upsample(scale_factor=scale_factor),   
      nn.ConvTranspose2d(32, 16, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.Upsample(scale_factor=scale_factor),
      nn.ConvTranspose2d(16, 8, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode),
      nn.BatchNorm2d(8),
      nn.ReLU(),
      nn.ConvTranspose2d(8, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode),
      nn.Upsample(scale_factor=scale_factor)
    )

    self.fusion = FusionLAyer()
    self.after_fusion = nn.Sequential(
      nn.Conv2d(1064,64, kernel_size=1),
      nn.ReLU()
    )

  def forward(self, input, embds):
    encoder = self.encoder(input)
    fusion = self.fusion(encoder, embds)
    after_fusion = self.after_fusion(fusion)
    # Upsample to get colors
    output = self.decoder(after_fusion)
    return output

In [32]:
model = ColorizationNet()

In [35]:
criterion = nn.MSELoss()

In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [37]:
if use_gpu: 
    criterion = criterion.cuda()
    model = model.cuda()

In [14]:
from torchsummary import summary

summary(model, [(1, 64, 64), (1,1000)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             160
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 32, 16, 16]           4,640
       BatchNorm2d-5           [-1, 32, 16, 16]              64
              ReLU-6           [-1, 32, 16, 16]               0
            Conv2d-7             [-1, 64, 8, 8]          18,496
       BatchNorm2d-8             [-1, 64, 8, 8]             128
              ReLU-9             [-1, 64, 8, 8]               0
      FusionLAyer-10           [-1, 1064, 8, 8]               0
           Conv2d-11             [-1, 64, 8, 8]          68,160
             ReLU-12             [-1, 64, 8, 8]               0
  ConvTranspose2d-13             [-1, 32, 8, 8]          18,464
      BatchNorm2d-14             [-1, 3

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [38]:
class AverageMeter(object):
  '''A handy class from the PyTorch ImageNet tutorial''' 
  def __init__(self):
    self.reset()
  def reset(self):
    self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
  def update(self, val, n=1):
    self.val = val
    self.sum += val * n
    self.count += n
    self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
  '''Show/save rgb image from grayscale and ab channels
     Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
  plt.clf() # clear matplotlib 
  color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
  color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
  color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
  color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
  color_image = lab2rgb(color_image.astype(np.float64))
  grayscale_input = grayscale_input.squeeze().numpy()
  if save_path is not None and save_name is not None: 
    plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
    plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [39]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'

In [40]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = AverageMeter()

    model.eval()
    already_saved_images = False
    for gray, ab, embd in val_loader:
        if use_gpu: 
            gray, ab, embd = gray.cuda(), ab.cuda(), embd.cuda()

        # Run model and record loss
        output_ab = model(gray, embd) # throw away class predictions
        loss = criterion(output_ab, ab)
        
        _loss.update(loss.item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print('Validate: Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
         loss=_loss))

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("Loss/test", _loss.avg, epoch)
    return _loss.avg

In [41]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = AverageMeter()
    
    model.train()

    for gray, ab, embd in train_loader:
        if use_gpu: 
            gray, ab, embd = gray.cuda(), ab.cuda(), embd.cuda()
            
        optimizer.zero_grad()
        output_ab = model(gray, embd)
        loss = criterion(output_ab, ab) 
        loss.backward()
        optimizer.step()
        
        _loss.update(loss.item(), gray.size(0))
        
    print(f'Epoch: {epoch}, Loss {_loss.val:.4f} ({_loss.avg:.4f})')
    
    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("Loss/train", _loss.avg, epoch)

In [42]:
# Make folders and set parameters
checkpoints = 'checkpoints'
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = 1e10
epochs = 250

In [43]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses < best_losses:
        best_losses = losses
        torch.save(model.state_dict(), '{}/model-epoch-{}-losses-{:.3f}.pth'.format(checkpoints,epoch+1,losses))

Starting training epoch 0
Epoch: 0, Loss 0.0076 (0.0221)
Finished training epoch 0


c:\Users\Franek\AppData\Local\Programs\Python\Python39\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 5 pixels
  return func(*args, **kwargs)
c:\Users\Franek\AppData\Local\Programs\Python\Python39\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 17 pixels
  return func(*args, **kwargs)


Validate: Loss 0.0077 (0.0075)	
Finished validation.
Starting training epoch 1
Epoch: 1, Loss 0.0074 (0.0075)
Finished training epoch 1


c:\Users\Franek\AppData\Local\Programs\Python\Python39\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 3 pixels
  return func(*args, **kwargs)


Validate: Loss 0.0078 (0.0075)	
Finished validation.
Starting training epoch 2
Epoch: 2, Loss 0.0079 (0.0072)
Finished training epoch 2
Validate: Loss 0.0073 (0.0070)	
Finished validation.
Starting training epoch 3
Epoch: 3, Loss 0.0072 (0.0070)
Finished training epoch 3
Validate: Loss 0.0081 (0.0077)	
Finished validation.
Starting training epoch 4
Epoch: 4, Loss 0.0071 (0.0068)
Finished training epoch 4
Validate: Loss 0.0073 (0.0071)	
Finished validation.
Starting training epoch 5
Epoch: 5, Loss 0.0069 (0.0067)
Finished training epoch 5
Validate: Loss 0.0076 (0.0074)	
Finished validation.
Starting training epoch 6
Epoch: 6, Loss 0.0063 (0.0066)
Finished training epoch 6
Validate: Loss 0.0077 (0.0075)	
Finished validation.
Starting training epoch 7
Epoch: 7, Loss 0.0077 (0.0065)
Finished training epoch 7


...

c:\Users\Franek\AppData\Local\Programs\Python\Python39\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 12 pixels
  return func(*args, **kwargs)


Validate: Loss 0.0086 (0.0082)	
Finished validation.
Starting training epoch 98
Epoch: 98, Loss 0.0046 (0.0039)
Finished training epoch 98
Validate: Loss 0.0081 (0.0076)	
Finished validation.
Starting training epoch 99
Epoch: 99, Loss 0.0039 (0.0039)
Finished training epoch 99
Validate: Loss 0.0082 (0.0082)	
Finished validation.
Starting training epoch 100
Epoch: 100, Loss 0.0041 (0.0039)
Finished training epoch 100
Validate: Loss 0.0079 (0.0078)	
Finished validation.
Starting training epoch 101
Epoch: 101, Loss 0.0039 (0.0039)
Finished training epoch 101
Validate: Loss 0.0081 (0.0080)	
Finished validation.
Starting training epoch 102
Epoch: 102, Loss 0.0042 (0.0039)
Finished training epoch 102
Validate: Loss 0.0086 (0.0082)	
Finished validation.
Starting training epoch 103
Epoch: 103, Loss 0.0037 (0.0039)
Finished training epoch 103
Validate: Loss 0.0079 (0.0079)	
Finished validation.
Starting training epoch 104
Epoch: 104, Loss 0.0040 (0.0038)
Finished training epoch 104
Validate: Lo

c:\Users\Franek\AppData\Local\Programs\Python\Python39\lib\site-packages\skimage\_shared\utils.py:394: UserWarning: Color data out of range: Z < 0 in 34 pixels
  return func(*args, **kwargs)


Validate: Loss 0.0097 (0.0090)	
Finished validation.
Starting training epoch 231
Epoch: 231, Loss 0.0035 (0.0035)
Finished training epoch 231
Validate: Loss 0.0087 (0.0081)	
Finished validation.
Starting training epoch 232
Epoch: 232, Loss 0.0034 (0.0035)
Finished training epoch 232
Validate: Loss 0.0086 (0.0082)	
Finished validation.
Starting training epoch 233
Epoch: 233, Loss 0.0038 (0.0034)
Finished training epoch 233
Validate: Loss 0.0096 (0.0089)	
Finished validation.
Starting training epoch 234
Epoch: 234, Loss 0.0034 (0.0035)
Finished training epoch 234
Validate: Loss 0.0087 (0.0085)	
Finished validation.
Starting training epoch 235
Epoch: 235, Loss 0.0033 (0.0035)
Finished training epoch 235
Validate: Loss 0.0088 (0.0082)	
Finished validation.
Starting training epoch 236
Epoch: 236, Loss 0.0034 (0.0034)
Finished training epoch 236
Validate: Loss 0.0087 (0.0081)	
Finished validation.
Starting training epoch 237
Epoch: 237, Loss 0.0034 (0.0035)
Finished training epoch 237
Valida

<Figure size 640x480 with 0 Axes>

In [44]:
torch.save(model.state_dict(), '{}/model-epoch-last-losses-{:.3f}.pth'.format(checkpoints,losses))

In [45]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: Loss 0.0085 (0.0079)	
Finished validation.


<Figure size 640x480 with 0 Axes>

In [ ]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()